In [116]:
import numpy as np
import pandas as pd
import jieba
import json
import jieba.analyse
from tqdm import tqdm
import re
from collections import Counter
import math

In [113]:
data = pd.read_csv(r'C:\Users\JDD\工作文件\近30日文章内容.csv')
test=pd.read_csv(r'C:\Users\JDD\工作文件\test_set_1225.csv',encoding='gbk')
test_data=test[['itemid','c_title','c_content']]
data=pd.concat([data,test_data])
test_list=list(test['itemid'].unique())
data=data.reset_index(drop=True)
data

,itemid,c_title,c_content
0,2130495,小忍者日本佛系跑山第二弹！夜跑神户市章山！【Moto vlog】,"[{""type"":""1"",""content"":""下一个视频应该就是飙车剪辑了""}]"
1,3609545,阳光明媚的一天,"[{""isEmptyForward"":false,""content"":"""",""id"":"""",..."
2,1636262,卖400万的摩托会飞？没错！它就是一台飞行摩托车，空陆两用,"[{""type"":""1"",""content"":""卖400万的摩托会飞？没错！它就是一台飞行摩..."
3,3594525,警钟长鸣 又一江西少年 过弯死亡！,"[{""content"":""关键词 超速驾驶 坠入路边菜地 不幸身亡"",""type"":""..."
4,869674,铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动,"[{""content"":""铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动..."
...,...,...,...
65150,3818063,高级定制改装哈雷机车,NaN
65151,3817180,BWM定制改装K1600B，够用科幻了吧，收藏了,"[{""content"":""这个改装车叫STELTH CROW。是以宝马K1600B为平台的改..."
65152,3817944,川崎中国发布ninja400/z400的召回信息。,"[{""content"":""几天前川崎在海外发布了400cc摩托车的召回信息，今天中午国内川崎..."
65153,3818304,张雪实测，凯越Cobra321R极速150km/h+,"[{""duration"":""73"",""a"":false,""b"":false,""img"":""h..."


In [108]:
def extractContent(data):
    if data not in ('', np.nan, None):
        content_list = json.loads(data)
        content=''
        if content_list not in ('', np.nan, None):
            for item in content_list:
                str_item = item.get('content')
                if str_item is not None:
                    content = content + str_item
        else:
            content=data
        return content
def clean_data(data):
    data=re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', data, flags=re.MULTILINE)
    data = data.replace('\n', '').replace('\xa0', '').replace('\u3000', ' ').replace('\u200B', ' ').replace('\r', '').replace('\t', '').replace('/', '，').replace('丨', '，').replace('！', '。').replace('？', '。').replace('：', '，').replace('~', '。').replace('.', '。').replace(',', '，').replace('-', '').replace(' ', '').lower()
    return data

In [114]:
data['content'] = data.c_content.apply(lambda x: extractContent(x))
data['content'].fillna('。',inplace=True)
data.drop(columns=['c_content'],inplace=True)
data['content'] = (data['c_title']+'。')*2+data['content']
data['content'] = data.content.apply(lambda x: clean_data(x))
data

,itemid,c_title,content
0,2130495,小忍者日本佛系跑山第二弹！夜跑神户市章山！【Moto vlog】,小忍者日本佛系跑山第二弹。夜跑神户市章山。【motovlog】。小忍者日本佛系跑山第二弹。夜...
1,3609545,阳光明媚的一天,阳光明媚的一天。阳光明媚的一天。
2,1636262,卖400万的摩托会飞？没错！它就是一台飞行摩托车，空陆两用,卖400万的摩托会飞。没错。它就是一台飞行摩托车，空陆两用。卖400万的摩托会飞。没错。它就...
3,3594525,警钟长鸣 又一江西少年 过弯死亡！,警钟长鸣又一江西少年过弯死亡。。警钟长鸣又一江西少年过弯死亡。。关键词超速驾驶坠入路边菜地不...
4,869674,铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动,铃木sfv650改装箭牌排气，声浪听着怎么样。有没有想驾驭它的冲动。铃木sfv650改装箭牌...
...,...,...,...
65150,3818063,高级定制改装哈雷机车,高级定制改装哈雷机车。高级定制改装哈雷机车。。
65151,3817180,BWM定制改装K1600B，够用科幻了吧，收藏了,bwm定制改装k1600b，够用科幻了吧，收藏了。bwm定制改装k1600b，够用科幻了吧，...
65152,3817944,川崎中国发布ninja400/z400的召回信息。,川崎中国发布ninja400，z400的召回信息。。川崎中国发布ninja400，z400的...
65153,3818304,张雪实测，凯越Cobra321R极速150km/h+,张雪实测，凯越cobra321r极速150km，h+。张雪实测，凯越cobra321r极速1...


In [117]:
jieba.analyse.set_stop_words(r'C:\Users\JDD\工作文件\stopwords.txt')
jieba.load_userdict(r'C:\Users\JDD\工作文件\user_dict.txt')
jieba.analyse.set_idf_path(r'C:\Users\JDD\工作文件\idf.txt')
#设置stop_words
stop_words = set((
        "the", "of", "is", "and", "to", "in", "that", "we", "for", "an", "are",
        "by", "be", "as", "on", "with", "can", "if", "from", "which", "you", "it",
        "this", "then", "at", "have", "all", "not", "one", "has", "or", "that"
    ))
stop_content = open(r'C:\Users\JDD\工作文件\stopwords.txt', 'rb').read().decode('utf-8')
for line in stop_content.splitlines():
    stop_words.add(line)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\JDD\AppData\Local\Temp\jieba.cache
Loading model cost 0.768 seconds.
Prefix dict has been built successfully.


In [118]:
allowPOS=['a','ad','ag','b','i','j','l','m','n','nr','ns','nt','nx','nz','s','r','t','tg','v','vd','vg','vn','x','z','eng']
#提取tf-idf关键词
def tf_idf_(data):
    tfidf_lst=jieba.analyse.extract_tags(data,topK=20,withWeight=True,allowPOS=allowPOS)
    tfidf_dct=dict(tfidf_lst)
    if len(tfidf_lst)>0:
        max_value=np.max(list(tfidf_dct.values()))
        tfidf_dct={x:math.sqrt(y/max_value) for x,y in tfidf_dct.items()}
    return tfidf_dct

#提取textrank关键词
def textrank_(data):
    txtrk=jieba.analyse.TextRank() 
    txtrk.span=10
    return txtrk.textrank(data,topK=20,withWeight=True,allowPOS=allowPOS)


#提取最终关键词
def extract_tokens(data):
    '''
    tfidf_keywords = tf_idf_(data)
    textrank_keywords = textrank_(data)
    tfidf_dict = dict(tfidf_keywords)
    textrank_dict = dict(textrank_keywords)
    key_words=tfidf_dict.keys()&textrank_dict.keys()
    keywords_dict ={}
    for word in key_words:
        keywords_dict[word] = tfidf_dict[word]*textrank_dict[word]
    tokens = sorted(keywords_dict.keys(),key=lambda x: x[1], reverse=True)
    '''
    tokens=list(dict(data).keys())
    return tokens

#提取带权重的关键词
def extract_keywords_weight(data):
    tfidf_dict = tf_idf_(data)
    textrank_keywords = textrank_(data)
    textrank_dict = dict(textrank_keywords)
    if len(tfidf_dict.keys())>0:
        max_value=np.max(list(tfidf_dict.values()))
        tfidf_dict={x:y/max_value for x,y in tfidf_dict.items()}
    if len(textrank_keywords)>0:
        max_value=np.max(list(textrank_dict.values()))
        textrank_dict={x:y/max_value for x,y in textrank_dict.items()}
    key_words=tfidf_dict.keys()&textrank_dict.keys()
    keywords_dict ={}
    for word in key_words:
        if word in tfidf_dict.keys():
            tfidf_value=tfidf_dict[word]
        else:
            tfidf_value=0
        if word in textrank_dict.keys():
            textrank_value=textrank_dict[word]
        else:
            textrank_value=0
        keywords_dict[word] = tfidf_value*textrank_value
    keywords_weight = sorted(keywords_dict.items(),key=lambda x: x[1], reverse=True)
    return keywords_weight

def get_keywords_weight(a,b):
    b=dict(b)
    if len(a.keys())>0:
        max_value=np.max(list(a.values()))
        a={x:y/max_value for x,y in a.items()}
    if len(b.keys())>0:
        max_value=np.max(list(b.values()))
        b={x:y/max_value for x,y in b.items()}
    key_words=a.keys()&b.keys()
    keywords_dict ={}
    for word in key_words:
        if word in a.keys():
            tfidf_value=a[word]
        else:
            tfidf_value=0
        if word in b.keys():
            textrank_value=b[word]
        else:
            textrank_value=0
        keywords_dict[word] = tfidf_value*textrank_value
    keywords_weight = sorted(keywords_dict.items(),key=lambda x: x[1], reverse=True)
    return keywords_weight

In [123]:
stcs='雅马哈 FORCE 155'
list(jieba.cut_for_search(stcs))

['马哈', '雅马哈', ' ', 'RC', 'FORCE', ' ', '15', '5']

In [124]:
data=pd.read_csv(r'F:\work\Downloads\query-hive-532571.csv')
data

,essay_id,embedding
0,186680,"[0.04376506301083379, -0.057889945057237796, -..."
1,265840,"[-0.07034701788598291, -0.13992218356156855, -..."
2,380960,"[0.03798328173770136, -0.09441575790177645, -0..."
3,458760,"[-0.11409198255189794, 0.10879582373814521, -0..."
4,563380,"[0.09133443870226249, 0.16666822627845573, -0...."
...,...,...
13988,3971018,"[-0.004240139726739625, -0.1473822589368498, -..."
13989,3971641,"[0.01927607092633302, -0.0364571610527618, -0...."
13990,3971567,"[0.07777082286213063, -0.08639653756500229, -0..."
13991,3971538,"[-0.08691066549577137, 0.07996363714086693, 0...."


In [125]:
effective_item_dict={}
for i in data.index:
    word=data['essay_id'][i]
    embedding=data['embedding'][i].replace('[','').replace(']','').split(', ')
    embedding=[float(x) for x in embedding]
    effective_item_dict[word]=embedding
print('有效文章向量存储成功')
effective_matrix=np.zeros((len(effective_item_dict.keys()),100))
effective_lst=list(effective_item_dict.keys())
for i in range(len(effective_lst)):
    effective_matrix[i]=np.array(effective_item_dict[effective_lst[i]])
print('有效文章向量矩阵存储成功')

有效文章向量存储成功
有效文章向量矩阵存储成功


In [146]:
lst=[x for x in effective_lst if x in act_lst]
len(lst)

10420

In [128]:
data=pd.read_csv(r'F:\work\Downloads\query-hive-532568.csv')
data

,essay_id,embedding
0,197427,"[-0.13929011028506547, 0.18919481660371762, -0..."
1,197947,"[0.053872537107650754, 0.345845547045104, -0.0..."
2,250087,"[-0.025019337304158062, 0.12707438733883777, -..."
3,261467,"[0.09464966903587471, 0.3197523950394005, -0.0..."
4,261707,"[-0.037963698182232476, 0.2818471033853573, 0...."
...,...,...
40481,3950599,"[-0.13045705976556043, 0.08562332621880825, -0..."
40482,3950639,"[0.1017462822525415, 0.17138477691362602, 0.10..."
40483,3950659,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
40484,3950739,"[0.018490827422260808, 0.1452171463516256, 0.0..."


In [131]:
data['essay_id'][0]

197427

In [129]:
act_item_dict={}
for i in data.index:
    word=data['essay_id'][i]
    embedding=data['embedding'][i].replace('[','').replace(']','').split(', ')
    embedding=[float(x) for x in embedding]
    act_item_dict[word]=embedding
print('有行为的文章向量存储成功')
act_matrix=np.zeros((len(act_item_dict.keys()),100))
act_lst=list(act_item_dict.keys())
for i in range(len(act_lst)):
    act_matrix[i]=np.array(act_item_dict[act_lst[i]])
print('有行为的文章向量矩阵存储成功')
sim_matrix=np.dot(act_matrix,effective_matrix.transpose())
print('相似度矩阵计算成功')

有行为的文章向量存储成功
有行为的文章向量矩阵存储成功
相似度矩阵计算成功


In [150]:
i=lst[0]
a=np.array(effective_item_dict[i])
b=np.array(act_item_dict[i])
c=np.dot(a,b.transpose())
c

-0.02365135545194393

In [152]:
b

array([ 0.11222677,  0.00588021,  0.04749319,  0.05563813, -0.03915023,
        0.17408812, -0.10402224, -0.07300053,  0.13685947, -0.10818321,
       -0.07116952,  0.00258747,  0.0790368 ,  0.01195663,  0.05982545,
       -0.04512615,  0.02542821,  0.10092009,  0.02991146,  0.10822006,
       -0.10577493,  0.08905274, -0.0642629 ,  0.21635273, -0.23932552,
        0.01335181,  0.04544619, -0.10485809, -0.07378905,  0.04728245,
        0.06147547, -0.04534405,  0.00470882,  0.11065715, -0.14004752,
        0.01204611, -0.12883101, -0.13965906, -0.07951461,  0.08420142,
       -0.04082863,  0.18837111,  0.05698769,  0.00286098,  0.02270186,
       -0.04317716, -0.06408266,  0.06275344, -0.04717173, -0.04672007,
       -0.20610362,  0.18694448, -0.21309313, -0.05199962,  0.05317078,
       -0.16329789,  0.02156648, -0.09840227, -0.01598156,  0.12153307,
       -0.13831063,  0.13202746,  0.02167682, -0.07814436, -0.23275031,
        0.04762917, -0.15449071,  0.12295642,  0.09846256, -0.11

In [144]:
sim_matrix[0]

-0.3128633806460808

In [163]:
output={}
for i in range(len(act_lst)):
    act_item=act_lst[i]
    essay1='kw_sim:'+str(act_item)
    output.setdefault(essay1,{})
    act_tensor=act_matrix[i]
    sim_items_all=list(np.sort(act_tensor))
    sim_items_all.reverse()
    sim_items=sim_items_all[:50]
    sim_args=list(np.argsort(-act_tensor))[:50]
    for j in range(len(sim_args)):
        arg=sim_args[j]
        sim_item=effective_lst[arg]
        sim_degree=sim_items[j]
        output[essay1][str(sim_item)]=sim_degree
print('相似度获取成功!')

相似度获取成功!


In [164]:
output

{'kw_sim:197427': {'2747100': 0.2093375264991195,
  '1504840': 0.20261222904425435,
  '265840': 0.18919481660371762,
  '1305220': 0.1847010811476504,
  '1040760': 0.18313952601765598,
  '940840': 0.17769173967103116,
  '563380': 0.1691960080005862,
  '671680': 0.15010259330446724,
  '1114360': 0.1475937402684486,
  '1920300': 0.13063661324445658,
  '1601780': 0.123946097636,
  '1365000': 0.11830341582683566,
  '1261360': 0.10707103440939396,
  '1027940': 0.1046031990530414,
  '1029780': 0.1043490307867722,
  '2755100': 0.10385527928945601,
  '1583520': 0.0829664275354664,
  '1273740': 0.08047841906630568,
  '774100': 0.07700031078688953,
  '2776100': 0.07689819921478469,
  '1559360': 0.07431020254820826,
  '1465320': 0.06823032305152885,
  '2739980': 0.06746541839695282,
  '1786300': 0.06564051951527027,
  '1018480': 0.065437115281686,
  '1252880': 0.06230015674272244,
  '983800': 0.06221306941578294,
  '1205740': 0.05077096683038288,
  '1274040': 0.04997910395399697,
  '1219480': 0.04

In [156]:
data = pd.read_csv(r'C:\Users\JDD\工作文件\近30日文章内容.csv')
data

,itemid,c_title,c_content
0,2130495,小忍者日本佛系跑山第二弹！夜跑神户市章山！【Moto vlog】,"[{""type"":""1"",""content"":""下一个视频应该就是飙车剪辑了""}]"
1,3609545,阳光明媚的一天,"[{""isEmptyForward"":false,""content"":"""",""id"":"""",..."
2,1636262,卖400万的摩托会飞？没错！它就是一台飞行摩托车，空陆两用,"[{""type"":""1"",""content"":""卖400万的摩托会飞？没错！它就是一台飞行摩..."
3,3594525,警钟长鸣 又一江西少年 过弯死亡！,"[{""content"":""关键词 超速驾驶 坠入路边菜地 不幸身亡"",""type"":""..."
4,869674,铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动,"[{""content"":""铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动..."
...,...,...,...
65100,2783852,没事洗洗...,"[{""type"":""1"",""content"":""""}]"
65101,2665627,焕新本色，多样生活！金城新款提拉米苏解析,"[{""type"":""1"",""content"":""金城的提拉米苏是一款200cc级欧系大轮踏板..."
65102,3694511,【4K】川崎机械增压街车Z H2 山路骑行第一视角,"[{""type"":""1"",""content"":""https://www.youtube.co..."
65103,2824784,慢慢骑~骑到老~骑在邯郸,"[{""type"":1,""content"":""感谢金球国际影院提供的免费场次！邯郸C1D大队~..."


,itemid,c_title,c_content
28451,1305220,骑着小破车去河边，星期四都有好几个在钓鱼。真闲,"[{""type"":""1"",""content"":""骑着小破车去河边，星期四都有好几个在钓鱼。真..."
